In [2]:
from os import getcwd

import pandas as pd
import os

In [3]:
FILE_NAME = "fifa_2014_all_players.csv"

path = f"{getcwd()}/data_raw/{FILE_NAME}"
df = pd.read_csv(path)

In [4]:
df

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,379165,Bailey WRIGHT,Defender,8,Preston North End FC,England,28.07.1992,21,184,Australia,0,0,False
732,369050,Ivan MOCINIC,Midfielder,15,HNK Rijeka,Croatia,30.04.1993,21,180,Croatia,0,0,True
733,380000,Marcelo BROZOVIC,Midfielder,14,GNK Dinamo Zagreb,Croatia,16.11.1992,21,180,Croatia,0,0,True
734,380009,Luis LOPEZ,Goalkeeper,1,Real Espana,Honduras,13.09.1993,20,182,Honduras,0,0,True


In [5]:
#check for missing values
missing_values = df.isnull().sum()
print(missing_values)

Player id                 0
Player                    0
Position                  0
Number                    0
Club                      0
Club (country)            0
D.O.B                     0
Age                       0
Height (cm)               0
Country                   0
Caps                      0
International goals       0
Plays in home country?    0
dtype: int64


In [6]:
#check for duplicates
def check_duplicates(df, columns=None):
    duplicate_count = df.duplicated(subset=columns).any()
    return duplicate_count
check_duplicates(df)


False

In [7]:
#check for trailing whitespace
def check_whitespaces(column):
    if column.dtype == "object":
        return column.str.contains(r'\s+$').any()
    return False
whitespaces = df.apply(check_whitespaces)
print(whitespaces)

Player id                 False
Player                    False
Position                  False
Number                    False
Club                      False
Club (country)            False
D.O.B                     False
Age                       False
Height (cm)               False
Country                   False
Caps                      False
International goals       False
Plays in home country?    False
dtype: bool


In [8]:
#convert column to correct data types (boolean)
df['Plays in home country?'] = df['Plays in home country?'].astype(bool)


In [9]:
#convert columns to correct data types (int)
df['Player id'] = df['Player id'].astype(int)
df['Number'] = df['Number'].astype(int)
df['Age'] = df['Age'].astype(int)
df['Height (cm)'] = df['Height (cm)'].astype(int)
df['Caps'] = df['Caps'].astype(int)
df['International goals'] = df['International goals'].astype(int)

In [10]:
#convert "D.O.B" to datetime
df['D.O.B'] = pd.to_datetime(df['D.O.B'], format='%d.%m.%Y')

In [11]:
#club node
clubs = df[['Club']].drop_duplicates()
clubs.rename(columns={'Club': 'Name'}, inplace=True)
clubs['ClubID'] = range(1,len(clubs)+1)
clubs['Index'] = clubs['ClubID']
clubs.set_index('Index',inplace=True)
clubs

,Name,ClubID
Index,,
1,Tigres UANL,1
2,Newcastle United Jets FC,2
3,Charlton Athletic FC,3
4,FC Barcelona,4
5,Galatasaray SK,5
...,...,...
293,VfR Aalen,293
294,Busan IPark FC,294
295,NK Lokomotiva Zagreb,295


In [12]:
#country node
# Combining the Country and Club (country) columns 
countries = pd.DataFrame(pd.concat([df['Country'], df['Club (country)']]), columns=['Country'])
countries = countries.drop_duplicates()
countries['CountryID']= range(1,len(countries)+1)
countries['Index'] = countries['CountryID']
countries.set_index('Index',inplace=True)
countries

,Country,CountryID
Index,,
1,Mexico,1
2,Australia,2
3,Iran,3
4,Brazil,4
5,Ivory Coast,5
6,Spain,6
7,Uruguay,7
8,Bosnia & Herzegovina,8
9,Netherlands,9


A new boolean property 'Played For National Team' is added. The property is derived from the Caps column, which indicates the number of times a player has represented their national team in international matches. This property is True for players who have at least one cap (indicating they have played for their national team) and False for players with zero caps.

In [13]:

#extract data
players = df[['Player id','Player','Position','Number','Club','D.O.B','Age','Club (country)','Height (cm)', 'Country', 'Caps','International goals', 'Plays in home country?']].drop_duplicates()
players.rename(columns={'Player id':'PlayerID','Player':'Name'}, inplace = True)

#Add new property 'Played For National Team'
players['Played For National Team'] = players['Caps'] > 0

# Transformation to convert player last names from "BOLD" to "Capitalized" 
make_name_capitalized = lambda full_name : " ".join([name.capitalize() for name in full_name.split(" ")])

# Applying transformation 
players["Name"] = players["Name"].apply(make_name_capitalized)

players['Index'] = players['PlayerID']
players.set_index('Index',inplace=True)
players

,PlayerID,Name,Position,Number,Club,D.O.B,Age,Club (country),Height (cm),Country,Caps,International goals,Plays in home country?,Played For National Team
Index,,,,,,,,,,,,,,
336722,336722,Alan Pulido,Forward,11,Tigres UANL,1991-03-08,23,Mexico,176,Mexico,5,4,True,True
368902,368902,Adam Taggart,Forward,9,Newcastle United Jets FC,1993-06-02,21,Australia,172,Australia,4,3,True,True
362641,362641,Reza Ghoochannejad,Forward,16,Charlton Athletic FC,1987-09-20,26,England,181,Iran,13,9,False,True
314197,314197,Neymar,Forward,10,FC Barcelona,1992-02-05,22,Spain,175,Brazil,48,31,False,True
212306,212306,Didier Drogba,Forward,11,Galatasaray SK,1978-03-11,36,Turkey,180,Ivory Coast,100,61,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379165,379165,Bailey Wright,Defender,8,Preston North End FC,1992-07-28,21,England,184,Australia,0,0,False,False
369050,369050,Ivan Mocinic,Midfielder,15,HNK Rijeka,1993-04-30,21,Croatia,180,Croatia,0,0,True,False
380000,380000,Marcelo Brozovic,Midfielder,14,GNK Dinamo Zagreb,1992-11-16,21,Croatia,180,Croatia,0,0,True,False


In [14]:
#player node
#drop columns
player = players.drop(['Club','Club (country)','Country','Caps','International goals','Plays in home country?','Played For National Team'], axis=1)
player

,PlayerID,Name,Position,Number,D.O.B,Age,Height (cm)
Index,,,,,,,
336722,336722,Alan Pulido,Forward,11,1991-03-08,23,176
368902,368902,Adam Taggart,Forward,9,1993-06-02,21,172
362641,362641,Reza Ghoochannejad,Forward,16,1987-09-20,26,181
314197,314197,Neymar,Forward,10,1992-02-05,22,175
212306,212306,Didier Drogba,Forward,11,1978-03-11,36,180
...,...,...,...,...,...,...,...
379165,379165,Bailey Wright,Defender,8,1992-07-28,21,184
369050,369050,Ivan Mocinic,Midfielder,15,1993-04-30,21,180
380000,380000,Marcelo Brozovic,Midfielder,14,1992-11-16,21,180


In [15]:
# Creating a table for the LOCATED_IN relationship
located_in_table = df[['Club','Club (country)']]


# Converts club name to club id
club_name_to_id = lambda name : clubs.index[clubs['Name'] == name][0]
# Converts country name to country id
country_name_to_id = lambda name : countries.index[countries['Country'] == name][0]


# Running the transformations 
located_in_table['Club'] = located_in_table['Club'].apply(club_name_to_id)
located_in_table['Club (country)'] = located_in_table['Club (country)'].apply(country_name_to_id)

# Renaming column
located_in_table.rename(columns={'Club (country)':'CountryID',
                                 'Club':'ClubID'}, 
                                 inplace = True)


located_in_table = located_in_table.drop_duplicates()


# Saving the LOCATED_IN relationship table as a csv
path_prefix = f'{getcwd()}/data_processed/'
located_in_table.to_csv(f'{path_prefix}rel_located_in.csv',index=False)

located_in_table

/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/2002892468.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located_in_table['Club'] = located_in_table['Club'].apply(club_name_to_id)
/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/2002892468.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located_in_table['Club (country)'] = located_in_table['Club (country)'].apply(country_name_to_id)
/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/2002892468.py:16

,ClubID,CountryID
0,1,1
1,2,2
2,3,17
3,4,6
4,5,33
...,...,...
723,293,12
725,294,26
729,295,21
731,296,17


In [16]:

# Creating a table for the REPRESENTS relationship
represents_table = players[['PlayerID','Country', 'Played For National Team', 'Caps','International goals']]
#Add new property 'Played For National Team'
players['Played For National Team'] = players['Caps'] > 0

# Converts country name to country id
country_name_to_id = lambda name : countries.index[countries['Country'] == name][0]

# Running the transformation 
represents_table['Country'] = represents_table['Country'].apply(country_name_to_id)

# Renaming columns
represents_table.rename(columns={'Player id':'PlayerID', 
                                 'Country':'CountryID'},
                                  inplace = True)

# Saving the REPRESENTS relationship table as a csv
path_prefix = f'{getcwd()}/data_processed/'
represents_table.to_csv(f'{path_prefix}rel_represents.csv',index=False)

represents_table

/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/3055311601.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  represents_table['Country'] = represents_table['Country'].apply(country_name_to_id)
/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/3055311601.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  represents_table.rename(columns={'Player id':'PlayerID',


,PlayerID,CountryID,Played For National Team,Caps,International goals
Index,,,,,
336722,336722,1,True,5,4
368902,368902,2,True,4,3
362641,362641,3,True,13,9
314197,314197,4,True,48,31
212306,212306,5,True,100,61
...,...,...,...,...,...
379165,379165,2,False,0,0
369050,369050,21,False,0,0
380000,380000,21,False,0,0


In [17]:
# Creating a table for the PLAYS_FOR relationship
plays_for_table = players[['PlayerID','Club', 'Plays in home country?']]

# Converts club name to club id
club_name_to_id = lambda name : clubs.index[clubs['Name'] == name][0]

# Running the transformation 
plays_for_table['Club'] = plays_for_table['Club'].apply(club_name_to_id)

# Renaming column
plays_for_table.rename(columns={'Player id':'PlayerID',
                                'Club':'ClubID'}, 
                                inplace = True)

# Saving the PLAYS_FOR relationship table as a csv
path_prefix = f'{getcwd()}/data_processed/'
plays_for_table.to_csv(f'{path_prefix}rel_plays_for.csv',index=False)

plays_for_table

/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/4268913180.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plays_for_table['Club'] = plays_for_table['Club'].apply(club_name_to_id)
/var/folders/gx/59q_qxd95tlg2ypf0nz73x5c0000gn/T/ipykernel_45348/4268913180.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plays_for_table.rename(columns={'Player id':'PlayerID',


,PlayerID,ClubID,Plays in home country?
Index,,,
336722,336722,1,True
368902,368902,2,True
362641,362641,3,False
314197,314197,4,False
212306,212306,5,False
...,...,...,...
379165,379165,296,False
369050,369050,297,True
380000,380000,23,True


In [18]:
# Saving the relationship tables as csvs

path_prefix = f'{getcwd()}/data_processed/'

clubs.to_csv(f'{path_prefix}clubs.csv',index=False)
countries.to_csv(f'{path_prefix}countries.csv',index=False)
player.to_csv(f'{path_prefix}players.csv',index=False)
